# Data Preprocessing



In [111]:
# utils
# transform two dict to feature vector
def dict_to_vec(dic1, dic2):
    kvec=[]
    vec1=[]
    for k, v in dic1.iteritems():
        kvec.append(k)
        # changed from counter to 1
        vec1.append(1)
    vec2=[0]*len(kvec)
    for k, v in dic2.iteritems():
        if k in kvec:
            index=kvec.index(k)
            if v==None:
                v=0
            vec2[index]=1
        else:
            kvec.append(k)
            vec1.append(0)
            vec2.append(1)
    return vec1, vec2

In [112]:
import re
import json
# validate the dict data for hero and item
def validate(data):
    id=[]
    for key in data:
        tid=data[key]
        id.append(tid)
    id.sort();
    flag=True
    for i in range(0, len(id)):
        if id[i]!=i:
            print i
            flag=False
    #print id
    return flag
# get the key-id pairs of hero or item data dict to be continuous
def continuous_format(data):
    i=0
    for key in data:
        data[key]=i
        i+=1

# construct item/hero-id based on json data
# reformat the id sequence
def dict_from_json(data, reindex):
    dict_data=dict()
    repeat=0
    for i in range(0, len(data)):
        #name=data[i]['name']
        name=re.sub('(npc_dota_hero_|item_)', '', data[i]['name'])
        #name=re.sub('[npc_|item_]', '', data[i]['name'])
        if name not in dict_data:
            if reindex:
                dict_data[name]= {
                    'id': (i-repeat),
                    'cost': data[i]['cost']
                }
            else:
                dict_data[name]=int(data[i]['id'])
        else:
            repeat+=1
            
        
    return dict_data

# for a up-to-date item/hero list, we can http request to
# https://api.opendota.com/api/explorer?sql=SELECT%20*%20FROM%20items
heroes=json.load(open('heroes.json'))['rows']
items=json.load(open('items.json'))['rows']

heroes=dict_from_json(heroes, False)
items=dict_from_json(items, True)

#if validate(heroes):
#    print "heroes continuous id assignment success!"
# if validate(items):
#     print "items continuous id assignment success!"
#print items
#print heroes
#print items

In [113]:
print items

{u'demon_edge': {'cost': 2200, 'id': 162}, u'recipe_refresher': {'cost': 1800, 'id': 52}, u'dagon_3': {'cost': 5215, 'id': 79}, u'recipe_medallion_of_courage': {'cost': 0, 'id': 118}, u'quarterstaff': {'cost': 875, 'id': 246}, u'belt_of_strength': {'cost': 450, 'id': 63}, u'recipe_ultimate_scepter': {'cost': 0, 'id': 85}, u'octarine_core': {'cost': 5900, 'id': 154}, u'aegis': {'cost': 0, 'id': 56}, u'recipe_cyclone': {'cost': 650, 'id': 43}, u'recipe_black_king_bar': {'cost': 1375, 'id': 55}, u'recipe_urn_of_shadows': {'cost': 310, 'id': 37}, u'medallion_of_courage': {'cost': 1175, 'id': 157}, u'recipe_mask_of_madness': {'cost': 0, 'id': 112}, u'flying_courier': {'cost': 150, 'id': 285}, u'recipe_rapier': {'cost': 0, 'id': 88}, u'tpscroll': {'cost': 50, 'id': 23}, u'recipe_veil_of_discord': {'cost': 500, 'id': 121}, u'nullifier': {'cost': 4700, 'id': 99}, u'claymore': {'cost': 1400, 'id': 237}, u'cloak': {'cost': 550, 'id': 1}, u'recipe_soul_booster': {'cost': 0, 'id': 104}, u'skadi': 

In [114]:
# based on the google drive data
# hero count=113
# item count=1020
import json
import os
import sys

fpath='./data/'

# init some coefficients
# the baseline freq coefficient
a=1.2
b=0.8

# init basic hero-item freq matrix

def maxValInDict(data):
    maxval=(-sys.maxint-1)
    for key in data:
        if data[key]>maxval:
            maxval=data[key]
    return maxval

def count_purchased_items(purchases, price_threshold=500):
    total_purchase_num = 0
    for item_name, num_purchased in purchases.items():
        if num_purchased is None:
            continue
            
        item_cost = items[item_name]['cost']
        if item_cost > price_threshold:
            total_purchase_num += num_purchased
    return total_purchase_num

hcount=maxValInDict(heroes)+1
icount=len(items)

hero_num_item_records = []

print "hero count: " + str(hcount)
print "item count: " + str(icount)
basic_freq=[]
for i in range(0, hcount):
    basic_freq.append([0]*icount)
    hero_num_item_records.append([])
    
num_records = len(os.listdir(fpath))

# load from data and computer base freq
# the first five heroes in the players list is radiant
for fname in os.listdir(fpath):
    data=json.load(open(fpath+fname))
    players=data['players']
    for i in range(0, len(players)):
        player=players[i]
        heroid=player['hero_id']
        purchase=player['purchase']
        
        num_purchased_items = count_purchased_items(purchase)
        hero_num_item_records[heroid].append(num_purchased_items)
        
        if player['isRadiant']:
            win=player['radiant_win']
        else:
            win=(not player['radiant_win'])
        # be noticed that purchase is a dictionary
        for key in purchase:
            iid=items[key]['id']
            try:
                if win: 
                    basic_freq[heroid][iid]+=a
                else: 
                    basic_freq[heroid][iid]+=b
            except IndexError as e:
                print "file name: " + fname
                print "item id: " + str(iid)
                print "hero id: " + str(heroid)

hero_item_count = []
for item_count_list in hero_num_item_records:
    if len(item_count_list) == 0:
        hero_item_count.append(0)
    else:
        hero_item_count.append(round(sum(item_count_list) / float(len(item_count_list))))
print hero_item_count

hero count: 121
item count: 292
[0, 19.0, 15.0, 12.0, 16.0, 9.0, 15.0, 8.0, 18.0, 15.0, 16.0, 19.0, 16.0, 13.0, 11.0, 14.0, 11.0, 21.0, 18.0, 13.0, 13.0, 14.0, 17.0, 14.0, 0, 16.0, 8.0, 11.0, 10.0, 16.0, 11.0, 11.0, 13.0, 12.0, 18.0, 17.0, 13.0, 9.0, 10.0, 14.0, 11.0, 17.0, 16.0, 13.0, 17.0, 14.0, 20.0, 15.0, 21.0, 15.0, 12.0, 14.0, 11.0, 16.0, 17.0, 13.0, 12.0, 16.0, 15.0, 15.0, 11.0, 9.0, 12.0, 16.0, 9.0, 15.0, 11.0, 17.0, 9.0, 12.0, 13.0, 14.0, 16.0, 15.0, 16.0, 12.0, 11.0, 12.0, 11.0, 10.0, 8.0, 14.0, 9.0, 11.0, 10.0, 14.0, 8.0, 7.0, 14.0, 10.0, 9.0, 10.0, 12.0, 15.0, 19.0, 17.0, 11.0, 10.0, 20.0, 18.0, 9.0, 12.0, 15.0, 11.0, 13.0, 17.0, 14.0, 7.0, 16.0, 16.0, 11.0, 7.0, 12.0, 9.0, 15.0, 0, 0, 0, 0, 12.0, 16.0]


In [115]:
# predict based on model
# the basic model is a matrix that contains weight 
# returns base h*k item recommendations
import random
from scipy import spatial as sp
import numpy as np
# if we do not consider time, rather we consider 
# all the necessary items in all time, then our method should also be effective
# in this case we may recommend more items like 15-20 that captures early-mid-final 
# k can be determined following:
# 1. get the avg total items throughout every match for every hero
# 2. guess
def base_rec(heroes, model, k):
    res=[]
    for h in heroes:
        hifreq=model[h]
        hifreq=hifreq.sort()
        r1=hifreq[0:k]
        r1=dict((r, 1) for r in r1)
        res.append(r1)
    return res

def base_rec_h(h, model, k):
    hifreq=model[h]
    hifreq=hifreq.sort()
    res=hifreq[0:k]
    res=dict((r, 1) for r in res)
    return res

# in matches won, recommend items, compute similarity with actual items
# OR
# in all games, find which side had items more similar to recommended, then compute winning percentage
# calculate the similarity between item purchase log
# per hero

# sim: similarity between item purchases
# ideal evaluation: P(Exactly same purchase log) 
# @hp: actual hero-purchase counter dict array
# @hp_rec: recommended hero-purchase counter dict array
# @opt: aggregation function, average and etc
# TODO: for diff heroes, we may have different weight when calc total similarity
def team_purchase_sim_calc(hp, hp_rec, norm=False, aggr_opt='avg'):
    sim_vec=[]
    tot_sim=0
    for (hp, hpr) in zip(hp, hp_rec):
        # item purchase counter to feature vector
        hp, hpr=dict_to_vec(hp, hpr)
        # do normalization if needed
        if norm:
            norm1=np.linalg.norm(hp)
            norm2=np.linalg.norm(hpr)
            hp=hp/norm1
            hpr=hpr/norm2
        # calc cosine similarity
        sim=sp.distance.cosine(hp, hpr)
        # append current hero item similarity
        sim_vec.append(sim)
    if aggr_opt=='avg':
        # the length should always be 5
        assert(len(hp)==5)
        tot_sim=sim_vec/len(hp)
    else:
        print "no such aggr function is pre defined!"
        tot_sim=-1
    return tot_sim

# assumes: we have tot_count[h] that stores the avg total "vital" item purchased by hero h
# assumes: dummy_is_vital(iid)
# necissity evaluation
# calculating probability 
def nec_eva(fpath, model, tot_count, pred=base_rec_h):
    for fname in os.listdir(fpath):
        data=json.load(open(fpath+fname))
        wplayers=[]
        # assumes: the first 5 is radiant hero
        # get the winner players
        if(data['radiant_win']):
            wplayers=data['players'][0:5]
        else:
            wplayers=data['players'][5:10]
        hero_vitem=[]
        rec_vitem=[]
        for p in wplayers:
            # vital items that we consider
            vitem=dict()
            purchase=p['purchase']
            hid=p['hero_id']
            for k in purchase:
                iid=items[k]
                if dummy_is_vital(iid):
                    vitem[k]=purchase[k]
            hero_vitem.append(vitem)
            base_rec_h(hid, base_freq, tot_count[hid])
        
        
    for i in range(0, len(players)):
        player=players[i]
        heroid=player['hero_id']
        purchase=player['purchase']
        if player['isRadiant']:
            win=player['radiant_win']
        else:
            win=(not player['radiant_win'])
# sufficiency evaluation
#def suf_eva():
    

In [116]:
# testing cosine similarity calculation
from scipy import spatial as sp
import numpy as np
data=json.load(open('data/3803000030.json'))
dict1=data['players'][0]['purchase']
dict2=data['players'][1]['purchase']

vec1, vec2=dict_to_vec(dict1, dict2)

print "unnormalized: "
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim

print "normalized: "
norm1=np.linalg.norm(vec1)
norm2=np.linalg.norm(vec2)
vec1=vec1/norm1
vec2=vec2/norm2
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim


unnormalized: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.582634993816
normalized: 
[ 0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.1796053  0.1796053  0.         0.         0.1796053  0.         0.
  0.1796053  0.         0.1796053  0.1796053  0.         0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053 

In [117]:
# testing cosine similarity calculation
from scipy import spatial as sp
import numpy as np
data=json.load(open('data/3803000030.json'))
dict1=data['players'][0]['purchase']
dict2=data['players'][1]['purchase']

vec1, vec2=dict_to_vec(dict1, dict2)

print "unnormalized: "
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim

print "normalized: "
norm1=np.linalg.norm(vec1)
norm2=np.linalg.norm(vec2)
vec1=vec1/norm1
vec2=vec2/norm2
print vec1
print vec2
sim=sp.distance.cosine(vec1, vec2)
print sim


unnormalized: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.582634993816
normalized: 
[ 0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889  0.25819889
  0.25819889  0.25819889  0.25819889  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
[ 0.1796053  0.1796053  0.         0.         0.1796053  0.         0.
  0.1796053  0.         0.1796053  0.1796053  0.         0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053  0.1796053
  0.1796053  0.1796053  0.1796053 

In [118]:
# evaluate difference between purchase_log and purchase
data=json.load(open('data/3803000030.json'))
exist=False
for p in data['players']:
    kdict=dict()
    for i in p['purchase_log']:
        key=i['key']
        if key in kdict:
            kdict[key]+=1
        else:
            kdict[key]=1
    print "purchase: " + str(len(p['purchase'])) + "\tlog: " + str(len(kdict))
    print "keys not in purchase log: "
    for key in p['purchase']:
        if key not in kdict:
            print key
    print ""
    #print "keys not in purchase: "
    #for key in kdict:
    #    if key not in p['purchase']:
    #        print key
    #        exist=True
#print exist

# TODO: evaluate if 0-4 is radiant, 5-9 is dire

purchase: 15	log: 15
keys not in purchase log: 

purchase: 31	log: 28
keys not in purchase log: 
dust
recipe_cyclone
recipe_hand_of_midas

purchase: 23	log: 21
keys not in purchase log: 
dust
recipe_yasha

purchase: 22	log: 18
keys not in purchase log: 
recipe_maelstrom
recipe_hand_of_midas
dust
recipe_diffusal_blade

purchase: 25	log: 22
keys not in purchase log: 
dust
recipe_diffusal_blade
recipe_yasha

purchase: 14	log: 12
keys not in purchase log: 
recipe_soul_ring
dust

purchase: 21	log: 21
keys not in purchase log: 

purchase: 37	log: 33
keys not in purchase log: 
dust
recipe_black_king_bar
recipe_sange
recipe_yasha

purchase: 38	log: 33
keys not in purchase log: 
recipe_manta
dust
recipe_wraith_band
recipe_yasha
recipe_force_staff

purchase: 19	log: 17
keys not in purchase log: 
recipe_force_staff
ward_dispenser



In [119]:
lst=[1, 2, 3, 54]
print lst[0:3]

[1, 2, 3]
